# Cours PCD – Labo 6 : Extraction d’attributs à partir de textes en vue de la classification

In [91]:
import numpy as np
import pandas as pd

from nltk.corpus import reuters
from sklearn.model_selection import GridSearchCV

## Exercice 1

Chargement des IDs nécéssaires

In [38]:
train_ids = [fid for fid in reuters.fileids() if fid.startswith("train")]
test_ids = [fid for fid in reuters.fileids() if fid.startswith("test")]

print(f"Nombre de fichiers d'entraînement: {len(train_ids)}")
print(f"Nombre de fichiers de test: {len(test_ids)}")
print(f"Nombre de fichiers total: {len(train_ids)+len(test_ids)}")

Nombre de fichiers d'entraînement: 7769
Nombre de fichiers de test: 3019
Nombre de fichiers total: 10788


## Exercice 2

Chargement des données dans des `DataFrame`:

In [81]:
def reuters_to_df(reuters, fids, category="grain"):
    return pd.DataFrame({
        "text" : map(lambda fid: " ".join(reuters.words(fid)), fids),
        "category" : map(lambda fid: 1 if category in reuters.categories(fid) else 0, fids)
    })

In [95]:
train = reuters_to_df(reuters, train_ids)
test = reuters_to_df(reuters, test_ids)

# Exercice 3

In [96]:
parameter_grid = {
    "vect__max_df": (0.2, 0.4, 0.6, 0.8, 1.0),
    "vect__min_df": (1, 3, 5, 10),
    "vect__ngram_range": ((1, 1), (1, 2)),  # unigrams or bigrams
    "vect__norm": ("l1", "l2"),
    "clf__alpha": np.logspace(-6, 6, 13),
}

In [97]:
random_search = GridSearchCV(
    estimator=pipeline,
    param_distributions=parameter_grid,
    n_iter=40,
    random_state=0,
    n_jobs=2,
    verbose=1,
)

NameError: name 'pipeline' is not defined